In [ ]:
import requests
import json

# API Key dari TMDb
api_key = '433164fc4dedec490c3816c120cf813e'  # Ganti dengan API key yang kamu dapatkan

In [ ]:

import pandas as pd
from datetime import datetime
import time 

# Hitung tahun saat ini dan 10 tahun yang lalu
current_year = datetime.now().year
start_year = current_year - 10

# Variabel untuk menyimpan semua film
all_movies = []
page = 1

# Loop untuk mengambil data film hingga mendapatkan minimal 5000 film
while len(all_movies) < 5000:
    # Endpoint untuk mengambil film selama 10 tahun terakhir
    url_movies = f'https://api.themoviedb.org/3/discover/movie?api_key={api_key}&primary_release_date.gte={start_year}-01-01&primary_release_date.lte={current_year}-12-31&language=en-US&page=1'
        
    # Mengirim request ke API TMDb untuk film
    response_movies = requests.get(url_movies)
    
    if response_movies.status_code == 200:
        data_movies = response_movies.json()
        movies = data_movies['results']

        if not movies:
            print("Semua halaman telah selesai diproses.")
            break
        
        # Tambahkan data film ke list
        all_movies.extend(movies)
        
        # Pindah ke halaman berikutnya
        page += 1
    else:
        print(f"Failed to retrieve movies: {response_movies.status_code}")
        break
        
# Buat DataFrame dari data film
df_movies = pd.DataFrame(all_movies, columns=['id', 'title', 'release_date', 'vote_average', 'genre_ids'])
    
# Endpoint untuk mengambil daftar genre
url_genres = f'https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}&language=en-US'
    
# Mengirim request ke API TMDb untuk genre
response_genres = requests.get(url_genres)
    
if response_genres.status_code == 200:
    data_genres = response_genres.json()
    genres = data_genres['genres']
    
    # Buat dictionary untuk mapping genre ID ke nama genre
    genre_dict = {genre['id']: genre['name'] for genre in genres}
        
    # Tambahkan kolom genre_name ke DataFrame film
    df_movies['genre_names'] = df_movies['genre_ids'].apply(lambda ids: [genre_dict[int(id)] for id in ids])
            
    # Hapus kolom genre_ids
    df_movies.drop(columns=['genre_ids'], inplace=True)
    
    # Tambahkan kolom untuk durasi dan deskripsi
    df_movies['duration'] = None
    df_movies['description'] = None
    
    # Loop untuk mendapatkan detail setiap film
    for idx, movie_id in enumerate(df_movies['id']):
        # Endpoint untuk mengambil detail film
        url_movie_details = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US'
        response_details = requests.get(url_movie_details)
    
        if response_details.status_code == 200:
            movie_details = response_details.json()
            # Menambahkan data durasi dan deskripsi ke DataFrame
            df_movies.at[idx, 'duration'] = movie_details.get('runtime')  # Durasi dalam menit
            df_movies.at[idx, 'description'] = movie_details.get('overview')  # Deskripsi film
        else:
            print(f"Failed to retrieve details for movie ID {movie_id}: {response_details.status_code}")
    
        # Rate limiting: tunggu 0.2 detik untuk mencegah blokir API
        time.sleep(0.2)
    
    # Simpan ke CSV
    df_movies.to_csv('tmdb.csv', index=False)
    print("Data film selama 10 tahun terakhir berhasil disimpan ke tmdb.csv")
else:
    print(f"Failed to retrieve genres: {response_genres.status_code}")
